<a href="https://colab.research.google.com/github/81300/AI-Notebooks/blob/testing/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternative PygmalionAI notebook for Google Colab

Version: GPU

This notebook is an *unofficial* method to run [PygmalionAI LLMs](https://github.com/PygmalionAI) (NSFW) that uses **oobabooga**'s [text-generation-webui project](https://github.com/oobabooga/text-generation-webui) in chat mode.

Run all the cells and a public gradio URL will appear at the bottom.

You can create your own custom characters using the [Pygmalion JSON character creator](https://oobabooga.github.io/character-creator.html). If you enable Google Drive you can also upload them to the following location:
  * [**Google Drive**](https://drive.google.com/) &rarr; **My Drive** &rarr; **Colab-TextGen** &rarr; **characters**

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

# From: https://github.com/henk717/KoboldAI
#@markdown Press play on the music player to keep the tab alive (Uses only 13MB of data).
%%html
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title 2. Clone the repository and install dependencies

Enable_Google_Drive = True #@param {type:"boolean"}

#@markdown After this cell completes you should click on the **Restart runtime** button if it appears, then move on to step 3.


if Enable_Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
else:
  import os
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

# Download Github repo
!git -C /content/drive/MyDrive/Colab-TextGen pull || git clone https://github.com/81300/text-generation-webui /content/drive/MyDrive/Colab-TextGen
%cd /content/drive/MyDrive/Colab-TextGen
!git switch autodevices

# Install requirements
%cd /content/drive/MyDrive/Colab-TextGen
!pip install --upgrade --no-cache-dir --progress-bar=off -r requirements.txt
!pip install --upgrade --no-cache-dir --progress-bar=off diffusers deepspeed
!pip install --upgrade --no-cache-dir --progress-bar=off Pillow==9.2.0
!DEBIAN_FRONTEND=noninteractive apt-get update
!DEBIAN_FRONTEND=noninteractive apt-get install -y libaio-dev

In [ ]:
#@title 3. Select a model

Model = "Pygmalion 6B" #@param ["Pygmalion 6B", "Pygmalion 6B experimental", "Pygmalion 2.7B", "Pygmalion 1.3B", "Pygmalion 350M"] {allow-input: false}
Reshard_model = True #@param {type: "boolean"}
Max_shard_size = "5GB" #@param ["10GB", "5GB", "3GB", "1GB"] {allow-input: false}
Model_storage = "Colab runtime" #@param ["Colab runtime", "Google Drive"] {allow-input: false}


if Model == "Pygmalion 6B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "main"
  model_name = "pygmalion-6b"
elif Model == "Pygmalion 6B experimental":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "dev"
  model_name = "pygmalion-6b_dev"
elif Model == "Pygmalion 2.7B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-2.7b"
  huggingface_branch = "main"
  model_name = "pygmalion-2.7b"
elif Model == "Pygmalion 1.3B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-1.3b"
  huggingface_branch = "main"
  model_name = "pygmalion-1.3b"
elif Model == "Pygmalion 350M":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-350m"
  huggingface_branch = "main"
  model_name = "pygmalion-350m"

import os
offload_path = "/content/offload"
if not os.path.exists(offload_path):
  os.makedirs(offload_path)
moodels_path = "/content/models"
if not os.path.exists(moodels_path):
  os.makedirs(moodels_path)

# Download and prepare model
%cd /content
![[ ! -f models/$model_name/config.json ]] && python /content/drive/MyDrive/Colab-TextGen/download-model.py $huggingface_org/$huggingface_repo --branch $huggingface_branch

if Reshard_model:
  import torch
  import diffusers
  from transformers import AutoTokenizer, AutoModelForCausalLM
  original_model = model_name
  resharded_model = model_name + f'_resharded'
  launch_model = resharded_model
  reshard_src_path = "/content/models/" + original_model
  reshard_out_path = "/content/models/" + resharded_model
  if not os.path.exists(reshard_out_path):
    os.makedirs(reshard_out_path)
  if os.path.exists(reshard_src_path):
    import gc
    gc.collect()
    torch.cuda.empty_cache()
    model = AutoModelForCausalLM.from_pretrained(reshard_src_path, torch_dtype="auto", low_cpu_mem_usage=True).cuda()
    tokenizer = AutoTokenizer.from_pretrained(reshard_src_path)
    model.save_pretrained(reshard_out_path, max_shard_size=Max_shard_size)
    tokenizer.save_pretrained(reshard_out_path)
else:
  launch_model = model_name

![[ "$Model_storage" = "Google Drive" ]] \
  && mv -f /content/models/$launch_model /content/drive/MyDrive/Colab-TextGen/models/$launch_model
![[ "$Model_storage" = "Colab runtime" ]] \
  && mkdir -p /content/drive/MyDrive/Colab-TextGen/models/$launch_model \
  && mount --bind /content/models/$launch_model /content/drive/MyDrive/Colab-TextGen/models/$launch_model

In [ ]:
#@title 4. Launch the web interface

%cd /content/drive/MyDrive/Colab-TextGen
import gc
gc.collect()
!LD_LIBRARY_PATH=/usr/local/cuda/lib64:/usr/lib64-nvidia python server.py --share --cai-chat --no-stream --model $launch_model

In [ ]:
#@title Cleanup all models

#@markdown Free up space within the current runtime. After this completes run step 4 again.

!find /content/models -mindepth 1 -maxdepth 1 -type d -exec rm -rv "{}" \;
!find /content/torch-dumps -mindepth 1 -exec rm -rv "{}" \;